In [1]:
import pandas as pd
df = pd.read_csv('preprocessed_lyrics.csv')


In [2]:
df

,SName,Lyric,Genre,Artist,lyric_length
0,Ask For More,['I mean I I livin know I just tell live hundr...,Hip Hop,Kevin Gates,530
1,Ridaz,['aw man yea you and people know jump you feel...,Hip Hop,Eminem,843
2,Blood On The Leaves,['Nina Simone strange hang poplar tree leave I...,Hip Hop,Kanye West,516
3,Don't Get Chipped,['can wait til I rich I finna buy gun from rou...,Hip Hop,Vince Staples,446
4,Fall In Love (Feat. Yg),['I wanna tell I I now I tryin tell exactly I ...,Hip Hop,Omarion,380
...,...,...,...,...,...
44383,Roll The Bones,['well stake good good winner loser seldom if ...,Rock,Rush,327
44384,Luxury,['Jagger Richards I want real fine fly Miami a...,Rock,Rolling Stones,265
44385,Vicious,['vicious hit you oh vicious vicious want hit ...,Rock,Lou Reed,210
44386,Since My Baby Left Me,['hey hey hey hey hey that nice the not miss s...,Rock,John Lennon,298


In [3]:
from sklearn.model_selection import train_test_split
 
train, test = train_test_split(df, test_size=0.2, random_state=42)
X_train = train.loc[:, train.columns != 'Genre']
y_train = train['Genre']
X_test = test.loc[:, test.columns != 'Genre']
y_test = test['Genre']

In [4]:
X_train

,SName,Lyric,Artist,lyric_length
44094,Telephone Bill,['well even I feel low I try help talk long af...,KT Tunstall,170
36183,Yesterday,['run scared look stop wait long nice clear sw...,Bad Religion,165
8021,All My Life,['all hustle all ride dirty all change all I w...,Flo Rida,691
4921,Happily Never After,['I think I want anymore as drop she say you l...,The Pussycat Dolls,387
30500,"If You Don't Like Me, Don't Look At Me",['run run young glen Run young glen see I care...,Morrissey,211
...,...,...,...,...
6265,Welcome To New York,['Jay turn motherfucking just Blaze you owe ni...,Jay-Z,325
11284,Remember The Name,['you ready lets yeah want know it like you co...,Fort Minor,664
38158,Stuck Without A Voice,['in clearheaded confused I think help choose ...,Hoobastank,240
860,That'z How It Iz,['keep thinkin easy like got to spit hot cop n...,Black Moon,545


In [5]:
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

class PairedSentenceVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    # Vectorize a single sentence.
    def _transform1(self, sentence):
        sentence = sentence.split()
        vector = np.array((sum(nlp.vocab[word].vector for word in sentence))/len(sentence))
        return (vector)

    # Vectorize a single row of the dataframe.
    def _transform2(self, row):
        # TODO: Replace the following line with your own code
        sentence = self._transform1(row.Lyric)
        return sentence

    def transform(self, X):
        return np.concatenate(
            [self._transform2(row).reshape(1, -1) for row in X.itertuples()]
        )

In [6]:
# TODO: Enter code here to implement the word embeddings classifier. Print the classification report.
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import spacy
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
from sklearn.neural_network import MLPClassifier
pipe = Pipeline(steps=[('vectorizer', PairedSentenceVectorizer()),
                       ('clf', MLPClassifier(random_state=1,  max_iter=8, hidden_layer_sizes=(300, 300, 300)))])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
target_names = df['Genre'].unique()
print(classification_report(y_test, y_pred, target_names=target_names))

/Users/martinfriberg/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

     Hip Hop       0.86      0.81      0.83      2980
         Pop       0.56      0.58      0.57      2903
        Rock       0.64      0.64      0.64      2995

    accuracy                           0.68      8878
   macro avg       0.68      0.68      0.68      8878
weighted avg       0.69      0.68      0.68      8878

